# Upload Specification file in each subject folder:
### From: Frink /HCP_Data/HCP_BedpostData/
### To: Candygram 
http://candygram.neurology.emory.edu:8080/#collection/5c48db6de62914004de6350f/folder/5c743889e62914004d6f974c

In [ ]:
# Ensure girder_client installed:
! pip install girder_client

In [1]:
import girder_client
import sys,os,glob
import numpy as np

In [2]:
# Subject Data location:
subjRootDir = "/data/HCP_BedpostData/"

# Source subject nodiff:
diffData = "/T1w/Diffusion/nodif*"

# Source ROIs
dtiSpaceMasks = "addlInfo/%s/DTI_ROIs/*"

In [3]:
# Admin login:
apiUrl="http://candygram.neurology.emory.edu:8080/api/v1"
cbGirder = girder_client.GirderClient(apiUrl=apiUrl)
cbGirder.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [4]:
# Destination of Subject data in DTI Space :
subjDTIPath = "/collection/FG/HCP_TractographyData/SubjectRegistrationResults/DTI_Space"
subjDtiTarget = cbGirder.get("resource/lookup?path=" + subjDTIPath)

In [5]:
# List of Subjects:
subjList = [x for x in os.listdir(subjRootDir) if 'addlInfo' not in x]
print(len(subjList),"Subjects have Bedpost Data")

1009 Subjects have Bedpost Data


# Define list of images in subject folder metadata:
dtiFilesForPapaya: Specifies load order of images in subject folder and subfolders

In [6]:
# DTI base data: first: nodif, second: nodif_brain, third: nodif_brain_mask
dtiFilesForPapaya = ['nodif.nii.gz',
                     'nodif_brain.nii.gz',
                     'nodif_brain_mask.nii.gz',
                     'Human_BA17_bilat_trans.nii.gz', 
                     'Human_BA_4ap_bilat_trans.nii.gz', 
                     'Human_BasalForebrain_Bilat_trans.nii.gz', 
                     'Human_BasalForebrain_Left_trans.nii.gz', 
                     'Human_BasalForebrain_Right_trans.nii.gz', 
                     'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 
                     'Human_Hypothalamus_Bilat_trans.nii.gz', 
                     'Human_Hypothalamus_Left_trans.nii.gz', 
                     'Human_Hypothalamus_Right_trans.nii.gz'
                    ]

In [7]:
# Iterate through subjects:
n=0
for s in subjList:
    print('S: %s' % s)
    
    # Reset variables:
    papayaSpecDataDict = {}
    papayaSpec={}
    imageNames = []
    imageList = []
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Images in subject folder:
    itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder[0]['_id']))
    
    # Subject subfolders: Ex. 'DTI_ROIs'
    foldersInSubjFolder = list(cbGirder.listFolder(grdrTgtSubjFolder[0]['_id']) )
    
    # Iterate through itemsInSubjFolder:
    for i in itemsInSubjFolder:   
        # Image location:                    
        imageURL = apiUrl +'/item/' +i['_id'] + '/download?contentDisposition=attachment'
        #print(imageURL)
        imageList.append( imageURL )
        # Image name:
        imageNames.append(i['name'])
        
    # Iterate through subfolders (Ex. 'DTI_ROIs'):
    for subFolder in foldersInSubjFolder:
        # List of subfolder images:
        subfolderImages = list(cbGirder.listItem(subFolder['_id']) )

        # Iterate through subfolder images:
        for subImage in subfolderImages:
            # Image location:
            imageURL = apiUrl +'/item/' +subImage['_id'] + '/download?contentDisposition=attachment'
            #print(imageURL)
            imageList.append( imageURL )
            # Image name:
            imageNames.append(subImage['name'])
            
    # Order imageList and iamgeNames according to dtiFilesForPapaya:
    indices=[]
    for i in imageNames:
        indices.append( dtiFilesForPapaya.index(i) )
    indices = np.argsort(indices)
        
    #print('imageNames: %s' % imageNames)
    #print('imageNames length: %s' % len(imageNames))
    #print('indices: %s' % indices)
    
    imageNames = [imageNames[x] for x in indices]
    imageList = [imageList[x] for x in indices]
    
    #print(imageNames)
    #print(imageList)
    
    # Upload metadata to Subject folder:        
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"imageList":imageList})
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"imageNames":imageNames})
    n=n+1
print('Total Subjects Updated: %s' % n)

S: 379657
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 112516
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 978578
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Ri

imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 206828
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Ri

S: 348545
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 103010
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

S: 101006
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 169747
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

...

S: 462139
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 103818
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

S: 173334
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 119126
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

S: 274542
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 362034
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 520228
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Ri

S: 429040
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypothalamus_Right_trans.nii.gz']
imageNames length: 12
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
S: 140319
imageNames: ['nodif.nii.gz', 'nodif_brain.nii.gz', 'nodif_brain_mask.nii.gz', 'Human_BA17_bilat_trans.nii.gz', 'Human_BA_4ap_bilat_trans.nii.gz', 'Human_BasalForebrain_Bilat_trans.nii.gz', 'Human_BasalForebrain_Left_trans.nii.gz', 'Human_BasalForebrain_Right_trans.nii.gz', 'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 'Human_Hypothalamus_Bilat_trans.nii.gz', 'Human_Hypothalamus_Left_trans.nii.gz', 'Human_Hypot

# Upload Specification file as metadata for each subject folder:
### Spec defines data visualized parameters and load order

In [ ]:
import json

In [ ]:
# Import specification file (randomise_output_aesthetic.js):
specFile = '../src/specification_randomise.json'

# Load json into variable:
with open(specFile, 'r') as f:
    specFileData = json.load(f)

In [ ]:
# Iterate through subjects:
for s in subjList:
    print('S: %s' % s)
    papayaSpecDataDict = {}
    papayaSpec={}
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Upload metadata to Subject folder:        
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"papayaSpec":specFileData})
    